In [1]:
# Starting in Section 10.3.3 - Module 10 exercises
# imports
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# set executable path and the URL for scraping
# Be sure to note Terms of Use for websites to see if automated browsing is permitted. Automated browsing can overload websites
# and some websites prohibit it.
executable_path={'executable_path': ChromeDriverManager().install()}
browser=Browser('chrome', **executable_path, headless=False)  # creates the automated browser window



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\sarah\.wdm\drivers\chromedriver\win32\91.0.4472.101]


In [ ]:
#Visit the mars NASA news site
url='https://redplanetscience.com'
browser.visit(url)
# Search for elmts with tag 'div' and attribute 'list_text'. Inspected website HTML and seeing div class="list_text"
# Optional delay for loading the page.
browser.is_element_present_by_css('div.list_text', wait_time=1)
